In [1]:
!pip install pymatgen
!pip install mp_api
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
from pymatgen.core.structure import Structure

In [2]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
import requests
import json
import os
import plotly.express as px


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset.xlsx",sheet_name='Export')

In [41]:
df = pd.read_excel("valence_electrons (3).xlsx")

In [42]:
df

,Unnamed: 0.1,Unnamed: 0,Perovskite,Hill formula,Interlayer space composition,Dopant,"Dopant, wt%","Bandgap, eV",DOI,Materials Project ID,...,c_MP,composition_obj,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons
0,0,0,K4Nb6O17,K4 Nb6 O17,NaN,NaN,NaN,3.50,10.1007/BF00765319,mp-560692,...,33.703336,K4 Nb6 O17,1.629630,2.518519,0.888889,0.000,0.323529,0.500000,0.176471,0.000000
1,1,1,KLaNb2O7,K La Nb2 O7,NaN,NaN,NaN,3.20,10.1007/BF00765319,mp-1223501,...,11.356399,K1 La1 Nb2 O7,1.727273,2.545455,0.818182,0.000,0.339286,0.500000,0.160714,0.000000
2,2,2,RbLaNb2O7,La Nb2 O7 Rb,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-553965,...,11.929214,La1 Nb2 O7 Rb1,1.727273,2.545455,0.818182,0.000,0.339286,0.500000,0.160714,0.000000
3,3,3,CsLaNb2O7,Cs La Nb2 O7,NaN,NaN,NaN,3.30,10.1007/BF00765319,mp-553248,...,11.503833,Cs1 La1 Nb2 O7,1.727273,2.545455,0.818182,0.000,0.339286,0.500000,0.160714,0.000000
4,4,4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-557195,...,15.190560,Ca2 K1 Nb3 O10,1.750000,2.500000,0.750000,0.000,0.350000,0.500000,0.150000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,519,519,Bi4Ti3O12,C8 Ca2 H20 N Nb3 O10,NaN,Pt,1.0,3.61,*,-1,...,NaN,C8 Ca2 H20 N1 Nb3 O10,1.477273,1.340909,0.272727,0.000,0.477941,0.433824,0.088235,0.000000
520,520,520,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.50,NaN,mp-557195,...,NaN,Ca2 K1 Nb3 O10,1.750000,2.500000,0.750000,0.000,0.350000,0.500000,0.150000,0.000000
521,521,521,KCa2Nb2TaO10,Ca2 K Nb2 O10 Ta,NaN,NaN,NaN,3.60,NaN,-1,...,NaN,Ca2 K1 Nb2 O10 Ta1,1.812500,2.500000,0.687500,0.875,0.308511,0.425532,0.117021,0.148936
522,522,522,KCa2NbTa2O10,Ca2 K Nb O10 Ta2,NaN,NaN,NaN,3.80,NaN,-1,...,NaN,Ca2 K1 Nb1 O10 Ta2,1.875000,2.500000,0.625000,1.750,0.277778,0.370370,0.092593,0.259259


In [43]:
from pymatgen.core.periodic_table import Element

def get_mulliken_en(element_symbol):
    el = Element(element_symbol)
    IE = el.ionization_energies[0]  # First ionization energy in eV
    EA = el.electron_affinity       # Electron affinity in eV

    if IE is None or EA is None:
        return None

    return (IE + EA) / 2

# Example
print(f"Mulliken EN of Oxygen: {get_mulliken_en('O')}")
print(f"Mulliken EN of Sodium: {get_mulliken_en('Na')}")

Mulliken EN of Oxygen: 7.5395839895
Mulliken EN of Sodium: 2.8435016049999997


In [44]:
def calc_average_electronegativity(formula):
  # Example: For Fe2O3
  comp = Composition(formula)
  # Weighted mean electronegativity
  total_atoms = comp.num_atoms
  mean_en = sum(
    #comp[el] / total_atoms * Element(el).X
    comp[el] / total_atoms * get_mulliken_en(el)
    for el in comp.elements)
  return mean_en

In [45]:
df["Average Mulliken electronegativity"] = df["Hill formula"].apply(calc_average_electronegativity)

In [46]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Perovskite,Hill formula,Interlayer space composition,Dopant,"Dopant, wt%","Bandgap, eV",DOI,Materials Project ID,...,composition_obj,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,Average Mulliken electronegativity
0,0,0,K4Nb6O17,K4 Nb6 O17,NaN,NaN,NaN,3.50,10.1007/BF00765319,mp-560692,...,K4 Nb6 O17,1.629630,2.518519,0.888889,0.0,0.323529,0.5,0.176471,0.0,5.958739
1,1,1,KLaNb2O7,K La Nb2 O7,NaN,NaN,NaN,3.20,10.1007/BF00765319,mp-1223501,...,K1 La1 Nb2 O7,1.727273,2.545455,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.994694
2,2,2,RbLaNb2O7,La Nb2 O7 Rb,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-553965,...,La1 Nb2 O7 Rb1,1.727273,2.545455,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.986554
3,3,3,CsLaNb2O7,Cs La Nb2 O7,NaN,NaN,NaN,3.30,10.1007/BF00765319,mp-553248,...,Cs1 La1 Nb2 O7,1.727273,2.545455,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.973029
4,4,4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-557195,...,Ca2 K1 Nb3 O10,1.750000,2.500000,0.750000,0.0,0.350000,0.5,0.150000,0.0,5.966812


In [9]:
fig = px.scatter(
    df,
    x='Average Mulliken electronegativity',                # x-axis data
    y='Bandgap, eV',                # y-axis data
    #y='Average electronegativity',
    title="Bandgap on electronegativity",     # Plot title
    labels={'x': "Average electronegativity", 'y': "Bandgap, eV"},  # Axis labels
    width=700,
    height=700,
)
fig.show()

In [10]:
!pip install matminer
from matminer.featurizers.composition import ValenceOrbital
from pymatgen.core import Composition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.9 MB/s eta 0:00:00


In [11]:
comp = Composition("K")
featurizer = ValenceOrbital()
features = featurizer.featurize(comp)

# Total valence electrons
total_valence = sum(features)
print(f"Total valence electrons (matminer): {total_valence}")

Total valence electrons (matminer): 2.0


/usr/local/lib/python3.11/dist-packages/matminer/featurizers/composition/orbital.py:115: UserWarning:

ValenceOrbital(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.

/usr/local/lib/python3.11/dist-packages/matminer/utils/data.py:326: UserWarning:

MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after f

In [47]:
df.to_excel("Average Mulliken electronegativity.xlsx")

#Valence electron dencity

In [48]:
comp = Composition("K4 Nb6 O17")
guess = comp.oxi_state_guesses()
guess

({'K': 1.0, 'Nb': 5.0, 'O': -2.0},)

In [49]:
import re
def split_formula(formula):
  output={}
  elements_with_indexes = formula.split()
  for el in elements_with_indexes:
    #match = re.match(r"([A-Za-z]+)(\d+)$", el)
    match = re.match(r"([A-Za-z]+)(\d+(?:\.\d+)?)$", el)
    if match:
      output[match.group(1)]=float(match.group(2))
    else:
      output[ el]=float(1)
  return output


In [50]:
def get_valence_electrons_number(hill_fomula):
  split = split_formula(hill_fomula)
  print(split)
  v = split.get("O")
  if v == None:
    return 0
  else:
    return 2*v

In [51]:
df["Valence electrons"] = df["Hill formula"].apply(get_valence_electrons_number)

{'K': 4.0, 'Nb': 6.0, 'O': 17.0}
{'K': 1.0, 'La': 1.0, 'Nb': 2.0, 'O': 7.0}
{'La': 1.0, 'Nb': 2.0, 'O': 7.0, 'Rb': 1.0}
{'Cs': 1.0, 'La': 1.0, 'Nb': 2.0, 'O': 7.0}
{'Ca': 2.0, 'K': 1.0, 'Nb': 3.0, 'O': 10.0}
{'Ca': 2.0, 'Nb': 3.0, 'O': 10.0, 'Rb': 1.0}
{'Ca': 2.0, 'Cs': 1.0, 'Nb': 3.0, 'O': 10.0}
{'K': 1.0, 'Nb': 3.0, 'O': 10.0, 'Sr': 2.0}
{'Ca': 2.0, 'K': 1.0, 'Na': 1.0, 'Nb': 4.0, 'O': 13.0}
{'O': 2.0, 'Ti': 1.0}
{'La': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Pr': 1.0, 'Rb': 1.0, 'Ta': 2.0}
{'Nd': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Rb': 1.0, 'Sm': 1.0, 'Ta': 2.0}
{'Ca': 1.0, 'O': 6.0, 'Ta': 2.0}
{'O': 6.0, 'Sr': 1.0, 'Ta': 2.0}
{'Ba': 1.0, 'O': 6.0, 'Ta': 2.0}
{'Bi': 2.0, 'Ca': 1.0, 'O': 9.0, 'Ta': 2.0}
{'Bi': 2.0, 'O': 9.0, 'Sr': 1.0, 'Ta': 2.0}
{'Ba': 1.0, 'Bi': 2.0, 'O': 9.0, 'Ta': 2.0}
{'Bi': 2.0, 'Ca': 1.0, 'Nb': 2.0, 'O': 9.0}
{'Bi': 2.0, 'Nb': 2.0, 'O': 9.0, 'Sr': 1.0}
{'Ba': 1.0, 'Bi': 2.0, 'Nb': 2.0, 'O': 9.0}
{'Bi': 2.0, 'Nb': 1.8, 'O': 9.0, 'Pb': 1.0

In [52]:
df.to_excel("valence_electrons.xlsx")

In [53]:
structure = Structure.from_file("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/mp-3249.cif")

In [ ]:
volume = structure.volume
print(f"Unit cell volume: {volume:.2f} Å³")

Unit cell volume: 238.91 Å³


In [54]:
def getCellVolume(MP_ID):
  if(MP_ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{MP_ID}.cif"
  if os.path.exists(file_path):
    structure = Structure.from_file(file_path)
  else:
    return 0

  if(structure == None):
    return 0
  return structure.volume

In [55]:
def getCellZValue(MP_ID):
  if(MP_ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{MP_ID}.cif"
  if os.path.exists(file_path):
    structure = Structure.from_file(file_path)
  else:
    return 0
  if(structure == None):
    return 0
  return structure.composition.num_atoms / structure.composition.reduced_composition.num_atoms

In [56]:
print(getCellZValue("mp-3249"))

1.0


In [57]:
df["Volume_MP"]=df["Materials Project ID"].apply(getCellVolume)

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.



In [58]:
df["MP_Z"]=df["Materials Project ID"].apply(getCellZValue)

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.



In [59]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Perovskite,Hill formula,Interlayer space composition,Dopant,"Dopant, wt%","Bandgap, eV",DOI,Materials Project ID,...,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,Average Mulliken electronegativity,Valence electrons,Volume_MP,MP_Z
0,0,0,K4Nb6O17,K4 Nb6 O17,NaN,NaN,NaN,3.50,10.1007/BF00765319,mp-560692,...,0.888889,0.0,0.323529,0.5,0.176471,0.0,5.958739,34.0,1767.044972,4.0
1,1,1,KLaNb2O7,K La Nb2 O7,NaN,NaN,NaN,3.20,10.1007/BF00765319,mp-1223501,...,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.994694,14.0,173.545022,1.0
2,2,2,RbLaNb2O7,La Nb2 O7 Rb,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-553965,...,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.986554,14.0,349.008754,2.0
3,3,3,CsLaNb2O7,Cs La Nb2 O7,NaN,NaN,NaN,3.30,10.1007/BF00765319,mp-553248,...,0.818182,0.0,0.339286,0.5,0.160714,0.0,5.973029,14.0,179.679908,1.0
4,4,4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-557195,...,0.750000,0.0,0.350000,0.5,0.150000,0.0,5.966812,20.0,924.746948,4.0


In [60]:
def calculateValenceElectronDencity(valenceElectrons, Z, Volume):
  if(Volume==0):
    return -1
  return valenceElectrons*Z/Volume


In [61]:
#df['Valence Electrons Density'] = calculateValenceElectronDencity(df["Valence electrons"], df["Z"], df["Volume"])
df['MP_Valence Electrons Density'] = (df['Valence electrons'] * df['MP_Z']) / df['Volume_MP']

In [62]:
df.to_excel("valence_electrons_density.xlsx")

In [25]:
fig = px.scatter(
    df,
    x='MP_Valence Electrons Density',                # x-axis data
    y='Bandgap, eV',                # y-axis data
    #y='Average electronegativity',
    title="Bandgap on Valence Electrons Density",     # Plot title
    labels={'x': "Valence Electrons Density", 'y': "Bandgap, eV"},  # Axis labels
    width=700,
    height=700,
)
fig.show()

In [63]:
def abcExtractionFromMP(MP_ID):
  if(MP_ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{MP_ID}.cif"
  if os.path.exists(file_path):
    structure = Structure.from_file(file_path)
  else:
    return 0

  if(structure == None):
    return 0
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]


In [ ]:
print(abcExtractionFromMP("mp-3249"))

[9.36623491, 9.36623491, 9.3662351]


In [64]:
df[['a_MP', 'b_MP', 'c_MP']] = df['Materials Project ID'].apply(abcExtractionFromMP).apply(pd.Series)

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.



In [28]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Perovskite,Hill formula,Interlayer space composition,Dopant,"Dopant, wt%","Bandgap, eV",DOI,Materials Project ID,...,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,Average Mulliken electronegativity,Valence electrons,Volume_MP,MP_Z,MP_Valence Electrons Density
0,0,0,K4Nb6O17,K4 Nb6 O17,NaN,NaN,NaN,3.50,10.1007/BF00765319,mp-560692,...,0.0,0.323529,0.5,0.176471,0.0,5.958739,34.0,1767.044972,4.0,0.076965
1,1,1,KLaNb2O7,K La Nb2 O7,NaN,NaN,NaN,3.20,10.1007/BF00765319,mp-1223501,...,0.0,0.339286,0.5,0.160714,0.0,5.994694,14.0,173.545022,1.0,0.080671
2,2,2,RbLaNb2O7,La Nb2 O7 Rb,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-553965,...,0.0,0.339286,0.5,0.160714,0.0,5.986554,14.0,349.008754,2.0,0.080227
3,3,3,CsLaNb2O7,Cs La Nb2 O7,NaN,NaN,NaN,3.30,10.1007/BF00765319,mp-553248,...,0.0,0.339286,0.5,0.160714,0.0,5.973029,14.0,179.679908,1.0,0.077916
4,4,4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.35,10.1007/BF00765319,mp-557195,...,0.0,0.350000,0.5,0.150000,0.0,5.966812,20.0,924.746948,4.0,0.086510


In [65]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
def getSymmetryGroup(MP_ID):
  if(MP_ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{MP_ID}.cif"
  if os.path.exists(file_path):
    structure = Structure.from_file(file_path)
  else:
    return 0

  if(structure == None):
    return 0
  spg = SpacegroupAnalyzer(structure)
  return spg.get_space_group_symbol()


In [83]:
print(getSymmetryGroup("mp-3249"))

R-3


In [75]:
df.to_excel("abc.xlsx")
df1=df

In [72]:
def extract_cif_from_COD(COD_ID):
  if(COD_ID==-1):
    return
  print(COD_ID)
  url = f"https://www.crystallography.net/cod/{COD_ID}.cif"
  response = requests.get(url)

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  if response.status_code == 200:
    print("Sucess")
    with open(f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{COD_ID}.cif', 'w') as f:
        f.write(response.text)
  else:
    print('No CIF')

In [78]:
df['COD_ID'].apply(extract_cif_from_COD)

1001842
Sucess
1545643
Sucess
2004917
Sucess
1521061
Sucess
2238958
Sucess
1518045
Sucess
1010942
Sucess
2002339
Sucess
1534928
Sucess
1537578
Sucess
1525923
Sucess
1521923
Sucess
1525921
Sucess
1531753
Sucess
7221049
Sucess
1531664
Sucess
1531664
Sucess
1531664
Sucess
1531664
Sucess
1531664
Sucess
1010942
Sucess
1533576
Sucess
1531664
Sucess
1530317
Sucess
1011028
Sucess
8101224
Sucess
7039788
Sucess
1526777
Sucess
1526720
Sucess
1526777
Sucess
8101224
Sucess
7039788
Sucess
1530317
Sucess
1001022
Sucess
1526777
Sucess
1011128
Sucess
2002850
Sucess
7039788
Sucess
1521923
Sucess
1528445
Sucess
1528445
Sucess
2106523
Sucess
2002850
Sucess
1010093
Sucess
1517788
Sucess
1517788
Sucess
1517788
Sucess
1517788
Sucess
1517788
Sucess
1517788
Sucess
1522043
Sucess
7221084
Sucess
1521427
Sucess
1528730
Sucess
2229871
Sucess
2229871
Sucess
2229872
Sucess
2229873
Sucess
2229874
Sucess
2229875
Sucess
7221013
Sucess
2102087
Sucess
1540895
Sucess
1530317
Sucess
1544411
Sucess
1528814
Sucess
7213033
Su

,COD_ID
0,None
1,None
2,None
3,None
4,None
...,...
519,None
520,None
521,None
522,None


In [79]:
def abcExtractionFromCOD(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    structure = Structure.from_file(file_path)
  else:
    return 0

  if(structure == None):
    return 0
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]

In [81]:
df[['a_COD', 'b_COD', 'c_COD']] = df['COD_ID'].apply(abcExtractionFromCOD).apply(pd.Series)

/usr/local/lib/python3.11/dist-packages/pymatgen/io/cif.py:1313: UserWarning:

Incorrect stoichiometry:
  CIF={'K': 1.0, 'La': 1.0, 'Nb': 2.0, 'O': 7.0}
  PMG={'K': 2.188, 'La': 1.936, 'Nb': 4.0, 'O': 14.0}
  ratios={'La': 1.936, 'Nb': 2.0, 'O': 2.0, 'K': 2.188}

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues 

In [84]:
df["Volume_COD"]=df["COD_ID"].apply(getCellVolume)

/usr/local/lib/python3.11/dist-packages/pymatgen/io/cif.py:1313: UserWarning:

Incorrect stoichiometry:
  CIF={'K': 1.0, 'La': 1.0, 'Nb': 2.0, 'O': 7.0}
  PMG={'K': 2.188, 'La': 1.936, 'Nb': 4.0, 'O': 14.0}
  ratios={'La': 1.936, 'Nb': 2.0, 'O': 2.0, 'K': 2.188}

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues 

In [85]:
df["Z_COD"]=df["COD_ID"].apply(getCellZValue)

/usr/local/lib/python3.11/dist-packages/pymatgen/io/cif.py:1313: UserWarning:

Incorrect stoichiometry:
  CIF={'K': 1.0, 'La': 1.0, 'Nb': 2.0, 'O': 7.0}
  PMG={'K': 2.188, 'La': 1.936, 'Nb': 4.0, 'O': 14.0}
  ratios={'La': 1.936, 'Nb': 2.0, 'O': 2.0, 'K': 2.188}

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3107: UserWarning:

Issues 

In [86]:
df['COD_Valence Electrons Density'] = (df['Valence electrons'] * df['Z_COD']) / df['Volume_COD']

In [87]:
df.to_excel("With_COD_data.xlsx")

In [98]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
def get_packing_fraction_from_cif(mp_id):
    cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
    cif_path = os.path.join(cif_folder, f"{mp_id}.cif")

    if not os.path.isfile(cif_path):
        return np.nan  # Return NaN if file doesn't exist
        #try:
    structure = Structure.from_file(cif_path)
    vire = ValenceIonicRadiusEvaluator(structure)
    print(structure.elements)
    print(vire.radii)
    print(vire.structure.lattice)
    #for site in structure:
    #  radius = vire.radii.get(site)
    #  print(radius)

    # Extract radius for each site
    #radii = [vire.radii[i] for i in range(len(structure)) if vire.radii[i] is not None]
    radii = []
    for site in structure:
      specie = site.specie
      if isinstance(specie, Specie):
        radius = specie.ionic_radius
      else:
        radius = None
      print(f"{site.species_string} → radius: {radius}")

    # Compute atomic volume per site: V = (4/3) * π * r³
    atom_volumes = [(4/3) * np.pi * r**3 for r in radii]
    total_atom_volume = sum(atom_volumes)

    # Compute packing fraction
    packing_fraction = total_atom_volume / structure.volume
    print("Packiging fraction ",packing_fraction)
    return packing_fraction
    #except:
        #return np.nan  # Return NaN if anything fails (e.g., missing radii)


In [99]:
print(get_packing_fraction_from_cif("mp-3249"))

[Element La, Element Ti, Element O]
{'La3+': 1.41, 'Ti4+': 0.88, 'O2-': 1.28}
5.345767 0.000000 7.690848
2.410346 4.771525 7.690848
0.000000 0.000000 9.366235


NameError: name 'Specie' is not defined

In [89]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Perovskite,Hill formula,Interlayer space composition,Dopant,"Dopant, wt%","Bandgap, eV",DOI,...,Valence electrons,Volume_MP,MP_Z,MP_Valence Electrons Density,a_COD,b_COD,c_COD,Volume_COD,Z_COD,COD_Valence Electrons Density
0,0,0,0,K4Nb6O17,K4 Nb6 O17,NaN,NaN,NaN,3.50,10.1007/BF00765319,...,34.0,1767.044972,4,0.076965,7.8300,33.20999,6.4600,1679.821072,4.0,0.080961
1,1,1,1,KLaNb2O7,K La Nb2 O7,NaN,NaN,NaN,3.20,10.1007/BF00765319,...,14.0,173.545022,1,0.080671,3.9060,21.60300,3.8879,328.066126,1.0,0.042674
2,2,2,2,RbLaNb2O7,La Nb2 O7 Rb,NaN,NaN,NaN,3.35,10.1007/BF00765319,...,14.0,349.008754,2,0.080227,0.0000,NaN,NaN,0.000000,0.0,NaN
3,3,3,3,CsLaNb2O7,Cs La Nb2 O7,NaN,NaN,NaN,3.30,10.1007/BF00765319,...,14.0,179.679908,1,0.077916,3.9080,3.90800,11.1600,170.440698,1.0,0.082140
4,4,4,4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.35,10.1007/BF00765319,...,20.0,924.746948,4,0.086510,3.8802,29.50800,7.7140,883.229408,4.0,0.090577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,519,519,519,Bi4Ti3O12,C8 Ca2 H20 N Nb3 O10,NaN,Pt,1.0,3.61,*,...,20.0,0.000000,0,NaN,0.0000,NaN,NaN,0.000000,0.0,NaN
520,520,520,520,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,NaN,NaN,3.50,NaN,...,20.0,924.746948,4,0.086510,3.8802,29.50800,7.7140,883.229408,4.0,0.090577
521,521,521,521,KCa2Nb2TaO10,Ca2 K Nb2 O10 Ta,NaN,NaN,NaN,3.60,NaN,...,20.0,0.000000,0,NaN,0.0000,NaN,NaN,0.000000,0.0,NaN
522,522,522,522,KCa2NbTa2O10,Ca2 K Nb O10 Ta2,NaN,NaN,NaN,3.80,NaN,...,20.0,0.000000,0,NaN,0.0000,NaN,NaN,0.000000,0.0,NaN
